<a href="https://colab.research.google.com/github/Shaik-arshiya-firdouse/plastic-type-classification/blob/main/plastic_classifier_from_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:
import os

# Path to your dataset folder in Google Drive
data_dir = "/content/drive/MyDrive/Plastic types"   # Change path if needed

# List all class folders
classes = os.listdir(data_dir)
print("Classes found:", classes)


Classes found: ['Others', 'PC', 'PE', 'PET', 'PP', 'PS']


In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input


In [ ]:
# Pretrained CNN for feature extraction
feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

def extract_features(img_path, target_size=(128,128)):
    try:
        img = load_img(img_path, target_size=target_size)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = feature_extractor.predict(img_array, verbose=0)  # shape (1, 1280)
        return features.flatten()
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None


/tmp/ipython-input-142810168.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  feature_extractor = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
X = []
y = []

for label, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, file)
        feat = extract_features(img_path)
        if feat is not None:
            X.append(feat)
            y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset shape:", X.shape, y.shape)


Dataset shape: (5, 1280) (5,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

params = {
    "objective": "multiclass",
    "num_class": len(classes),
    "metric": "multi_logloss",
    "boosting_type": "gbdt"
}

model = lgb.train(
    params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=100,
    callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(10)]
)



[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 4, number of used features: 0
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [War

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred_classes))


Accuracy: 1.0


In [ ]:
# Test with a new image from Drive
test_img = "/content/drive/MyDrive/Plastic types/image_101.jpg"   # your image path

features = extract_features(test_img).reshape(1, -1)
prediction = np.argmax(model.predict(features), axis=1)

print("Predicted class:", classes[prediction[0]])



Predicted class: PE
